In [2]:
# Install required packages and set up Chrome WebDriver
!pip install selenium beautifulsoup4
!apt-get update
!apt-get -q -y install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install webdriver_manager

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [983 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,136 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-securi

In [3]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [4]:
def scrape_reviews(url, driver):
    driver.get(url)
    time.sleep(2)

    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@class, 'btn-group-btn') and contains(@class, 'next')]"))
        )
        next_button.click()
    except (TimeoutException, NoSuchElementException, StaleElementReferenceException):
        return None

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    review_elements = soup.find_all("div", class_=lambda x: x and x.startswith("topic-reply"))

    return review_elements


In [7]:
# Initialize Chrome WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # To run in headless mode
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)

# Initialize empty lists to store the data
names = []
comments = []
dates = []

page = 1
stop = False  # Initialize the 'stop' variable

while not stop:
    url = f"https://www.pistonheads.com/gassing/topic.asp?h=0&f=23&t=2033596&i={page * 20}"

    driver.get(url)
    time.sleep(2)

    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@class, 'btn-group-btn') and contains(@class, 'next')]"))
        )
        next_button.click()
    except (TimeoutException, NoSuchElementException, StaleElementReferenceException):
        stop = True  # Set 'stop' to True to exit the loop
        break

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    review_elements = soup.find_all("div", class_=lambda x: x and x.startswith("topic-reply"))

    if len(review_elements) == 0:
        stop = True
        break

    for review in review_elements:
        name_element = review.find("p", class_="author js-author")
        comment_element = review.find("div", class_="phml msg-body")
        date_element = review.find("span", class_="timestamp")

        name = name_element.text.strip() if name_element else ""
        comment = comment_element.text.strip() if comment_element else ""
        date = date_element.text.strip() if date_element else ""

        names.append(name)
        comments.append(comment)
        dates.append(date)

    page += 1

In [8]:
# Close the webdriver
driver.quit()

In [10]:
# Create a pandas DataFrame from the scraped data
pistonheads2 = pd.DataFrame({
    "Name": names,
    "Comment": comments,
    "Date": dates
})

In [11]:
# Display the review data
print(pistonheads2)

             Name                                            Comment  \
0        ecsrobin  Just for those thinking the price is high:Hond...   
1       dvshannow  ecsrobin said:\n Just for those thinking the p...   
2       skedaddle  SpadeBrigade said:\n I know it was a LONG time...   
3    Cornish Pete  If I was forced I might just pay £24-£25k for ...   
4        ecsrobin  dvshannow said:\n GR Yaris 35kAs an owner of a...   
..            ...                                                ...   
98         V8Head  One word, RANGE!I won’t consider any electric ...   
99       ecsrobin  ChrisH72 said:\n Yaris GR. 35k new, still 30k ...   
100  Julian Scott  V8Head said:\n One word, RANGE!I won’t conside...   
101         D4rez  Julian Scott said:\n Ironic from the driver of...   
102       nismo48  hozepipe said:\n I'll stick with my 12 YO 695 ...   

                   Date  
0      Thursday 4th May  
1      Thursday 4th May  
2      Thursday 4th May  
3      Thursday 4th May  
4    

In [12]:
# Save the DataFrame to a CSV file
csv_filename = "pistonheads2.csv"  # You can choose the desired file name
pistonheads2.to_csv(csv_filename, index=False)

In [13]:
# Display a message after saving
print(f"Scraped data saved to {csv_filename}")

Scraped data saved to pistonheads2.csv


In [14]:
import os

In [15]:
# Define the folder name
folder_name = "Colab_Data"

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
# Create the folder in Google Drive
drive_path = "/content/drive/MyDrive/"
folder_path = os.path.join(drive_path, folder_name)

In [18]:
# Create the folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

print(f"Folder '{folder_name}' created in Google Drive")

Folder 'Colab_Data' created in Google Drive


In [19]:
csv_filename = os.path.join(folder_path, "pistonheads2.csv")
review_data.to_csv(csv_filename, index=False)

print(f"CSV file saved to '{csv_filename}'")

CSV file saved to '/content/drive/MyDrive/Colab_Data/pistonheads2.csv'
